### calculate the SA signal for each order

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii, fits
from matplotlib.gridspec import GridSpec
from scipy import interpolate
from astropy.visualization import MinMaxInterval, AsinhStretch, HistEqStretch, ImageNormalize
from scipy.optimize import curve_fit
%matplotlib inline
#%matplotlib ipympl

In [2]:
# load atmospheric transmission file
hdu = fits.open('/Users/jpw/idl/Spextool/data/atran75000.fits')
tdata = hdu[0].data
atrans = interpolate.interp1d(tdata[0,:], tdata[1,:])
hdu.close()

In [3]:
j1 = 30           # starting point of first order -- figured out by hand
dj_AB = 121       # width of each order -- this should match the size of the number of rows in the order extension
dj_blank = 30     # gap between orders -- this is figured out by eye and assumed to be the same for all orders

In [4]:
# two gaussians with different peaks and offsets but same FWHM
# added an extra constant offset since some orders are not perfectly sky-subtracted
def gauss2(x, A1, x1, A2, x2, fwhm, C):
    sigma = fwhm / (8*np.log(2))
    y = A1 * np.exp(-0.5*((x-x1)/sigma)**2) + A2 * np.exp(-0.5*((x-x2)/sigma)**2) + C
    return y

In [5]:
def fit_median(order_flux, pix_nofit=0, plotfile=None):
    # stack an order in wavelength to measure the average slit profile and guide individual fits
    im_median = np.nanmedian(order_flux, axis=1)
    j = np.arange(im_median.size)
    p0 = [np.min(im_median), np.argmin(im_median), np.max(im_median), np.argmax(im_median), 2, 0]

    # don't fit within s_nofit of the edges
    fit_range = (j > pix_nofit) & (j < j[-1]-pix_nofit)

    # fit two gaussians to the data
    pfit, pcov = curve_fit(gauss2, j[fit_range], im_median[fit_range], p0)
    #perr = np.sqrt(np.diag(pcov))

    if plotfile is not None:
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.step(j, im_median)
        ax.plot(j[fit_range], gauss2(j[fit_range], *pfit))
        ax.set_xlabel(r"Row (pixels)", fontsize=14)
        ax.set_ylabel(r"Flux (Jy)", fontsize=14)
        fig.tight_layout()
        fig.savefig(plotfile, dpi=300)

    return pfit

In [6]:
def calculate_SA(order_flux, pix_nofit=0, nfwhm=1, verbose=False):
    ny, nx = order_flux.shape
    pfit = fit_median(order_flux, pix_nofit=pix_nofit)#, plotfile='median_profile.png')

    jneg =  pfit[1] + np.array([0, -0.5*nfwhm, 0.5*nfwhm]) * pfit[4]
    jpos =  pfit[3] + np.array([0, -0.5*nfwhm, 0.5*nfwhm]) * pfit[4]
    SA  = np.zeros((4, nx)) + np.nan

    for i in range(nx):
        flux_slice = order_flux[:, i]
        #weights = 1 / order_var[:, i]

        jsum1 = 0.
        jsum2 = 0.
        fsum = 0.
        for j1 in range(int(jneg[1]+0.5), int(jneg[2]+1.5)):
            f1 = flux_slice[j1]
            jsum1 += (j1-jneg[0]) * f1
            jsum2 += (j1-jpos[0])**2 * f1
            fsum += f1
        jmean = jsum1 / fsum
        SA[0, i] = jmean
        SA[2, i] = np.sqrt((jsum2 / fsum - jmean**2) / (jneg[2] - jneg[1] + 1))

        jsum1 = 0.
        jsum2 = 0.
        fsum = 0.
        for j1 in range(int(jpos[1]+0.5), int(jpos[2]+1.5)):
            f1 = flux_slice[j1]
            jsum1 += (j1-jpos[0]) * f1
            jsum2 += (j1-jneg[0])**2 * f1
            fsum += f1
        jmean = jsum1 / fsum
        SA[1, i] = jmean
        SA[3, i] = np.sqrt((jsum2 / fsum - jmean**2) / (jpos[2] - jpos[1] + 1))

    return SA

In [7]:
# path to the directory containing the combined rectified and cal files
#path = '/Volumes/JPW_4TB/iSHELL/191008/'
#path = '/Volumes/JPW_4TB/iSHELL/200221/'
path = '/Volumes/JPW_4TB/iSHELL/200625/'

# read in the source and wavecal from the manually edited sourcefile.txt
with open(path+'rectified/'+'sourcelist.txt') as f:
    all_lines = f.read()
lines = all_lines.split('\n')

# find the breakpoints between sources
nbreak = []
for nline, line1 in enumerate(lines):
    if line1[0:4] == '----':
        nbreak.append(nline)

# loop through the sources
print('-'*40)
for n1 in range(len(nbreak)-1):
    source = lines[nbreak[n1]+1]
    calfile = lines[nbreak[n1]+2]

    hdu1 = fits.open(path+'reduced/'+source+'_rectified.fits')
    flux = hdu1[0].data
    #var = hdu1[1].data
    nslit, nwl = flux.shape

    hdu2 = fits.open(path+'cal/'+calfile)
    wc_hd = hdu2[0].header
    orders = wc_hd['ORDERS'].split(',')

    csvfile = open(path+'reduced/'+source+'_SA.csv', 'w')
    csvfile.write('wavelength,  off_neg, off_pos, err_neg, err_pos\n')

    # loop through the orders in reverse order so that the wavelength monotonically increases
    print(source)
    for n, order in enumerate(orders[::-1]):
        norder = len(orders) - n - 1
        j0 = j1 + (dj_AB + dj_blank) * norder

        wavecal = hdu2[3+norder].data
        wl0 = wavecal[0, 0, 1:]
        # pad the wavelength file for order 99 where the order cuts off before the end
        wl = np.pad(wl0, (0, nwl-wl0.size), 'edge')
        print(f'Order = {order}, Min/Max wavelength = {wl.min()}, {wl.max()}')

        SA = calculate_SA(flux[j0:j0+dj_AB, :], pix_nofit=5, nfwhm=1)
        flag = atrans(wl) < 0.5
        SA[:, flag] = np.nan

        for i in range(wl.size):
            csvfile.write(f'{wl[i]:11.9f}, {SA[0, i]:7.4f}, {SA[1, i]:7.4f}, {SA[2, i]:7.4f}, {SA[3, i]:7.4f}\n')

    print('-'*40)
    hdu1.close()
    hdu2.close()
    csvfile.close()
    

----------------------------------------
DFTau_PA138
Order = 114, Min/Max wavelength = 4.5092653174986355, 4.547357707125535
Order = 113, Min/Max wavelength = 4.549027885058983, 4.5874563449482935
Order = 112, Min/Max wavelength = 4.58950038440199, 4.628275002413277


/var/folders/tx/plmj8k8969gcvbqj4tml5hvm0000gn/T/ipykernel_45810/2368970709.py:35: RuntimeWarning: invalid value encountered in sqrt
  SA[3, i] = np.sqrt((jsum2 / fsum - jmean**2) / (jpos[2] - jpos[1] + 1))
/var/folders/tx/plmj8k8969gcvbqj4tml5hvm0000gn/T/ipykernel_45810/2368970709.py:23: RuntimeWarning: invalid value encountered in sqrt
  SA[2, i] = np.sqrt((jsum2 / fsum - jmean**2) / (jneg[2] - jneg[1] + 1))


Order = 111, Min/Max wavelength = 4.630702002882935, 4.669833139520757
Order = 110, Min/Max wavelength = 4.672652625579461, 4.712150923907736
Order = 109, Min/Max wavelength = 4.715372867297214, 4.755249263308378
Order = 108, Min/Max wavelength = 4.758884106353559, 4.799149839817777
Order = 107, Min/Max wavelength = 4.803208520255721, 4.843875146077061
Order = 106, Min/Max wavelength = 4.848369123398432, 4.889448523506733
Order = 105, Min/Max wavelength = 4.894389806915752, 4.9358942027247386
Order = 104, Min/Max wavelength = 4.941295380832006, 4.983237346296379
Order = 103, Min/Max wavelength = 4.989111618668148, 5.0315040939745
Order = 102, Min/Max wavelength = 5.037865304671995, 5.080721610600852
Order = 101, Min/Max wavelength = 5.087584283854259, 5.130918136853117
Order = 100, Min/Max wavelength = 5.138297515026703, 5.1821230430367216
Order = 99, Min/Max wavelength = 5.190035127054771, 5.233705273566629
----------------------------------------
HR1473_PA138
Order = 114, Min/Max wav